# FIR MMSE-LE equalization

In [1]:
import numpy as np

In [85]:
Nf = 4
delta = 0
v = 1
epsilon = 1
sigma2 = 0.97/(10**1.2)
Rnn = np.identity(Nf)

In [86]:
# create P matri

P = np.array([[0.9, -0.4, 0, 0, 0], [0, 0.9, -0.4, 0, 0], [0, 0, 0.9, -0.4, 0], [0, 0, 0, 0.9, -0.4]])
print(P)
print("Shapes match:",np.shape(P) == (Nf, Nf + v))

[[ 0.9 -0.4  0.   0.   0. ]
 [ 0.   0.9 -0.4  0.   0. ]
 [ 0.   0.   0.9 -0.4  0. ]
 [ 0.   0.   0.   0.9 -0.4]]
Shapes match: True


In [94]:
# computing matrix to find the optimal delta
M = np.dot(P.T, np.dot(np.linalg.inv(Rnn), P)) + (sigma2/epsilon)*np.identity(Nf + v)
print(np.linalg.inv(M).diagonal())

[ 1.38923578  1.41372959  1.68546487  3.34711226 13.38604448]


In [88]:
# find Ryy
ryy = (epsilon*np.dot(P,P.T) + sigma2*Rnn)
ryy

array([[ 1.03120286, -0.36      ,  0.        ,  0.        ],
       [-0.36      ,  1.03120286, -0.36      ,  0.        ],
       [ 0.        , -0.36      ,  1.03120286, -0.36      ],
       [ 0.        ,  0.        , -0.36      ,  1.03120286]])

In [89]:
# finding rx
rx = epsilon*P[:, delta]
rx = rx[np.newaxis, :]
print(rx)

[[0.9 0.  0.  0. ]]


In [90]:
# finding w
w = np.dot(rx, np.linalg.inv(ryy))
print(w)

[[1.01663866 0.41211304 0.16383954 0.05719751]]


In [91]:
# finding sigma2_MMSE-LE
sigma_mmse = epsilon - np.dot(w,rx.T)
print(sigma_mmse)

[[0.08502521]]


In [92]:
# finding coefficients of x_ks
xk_coeffs = np.dot(w,P)
print(xk_coeffs)
print("Maximum gain for x_{k-delta}:", np.argmax(xk_coeffs) == delta)

[[ 0.91497479 -0.03575372 -0.01738964 -0.01405806 -0.022879  ]]
Maximum gain for x_{k-delta}: True


In [93]:
# finding SNR
signal_power = xk_coeffs[0,delta]**2
noise_power = sigma2*np.sum(w**2)
isi_power = np.sum(xk_coeffs**2) - signal_power
SNR_b = signal_power/(isi_power + noise_power)
print("SNR_biased:", SNR_b)
print("SNR_biased (dB)", 10*np.log10(SNR_b))

signal_power = 1
isi_power = isi_power + (1 - xk_coeffs[0,delta])**2
SNR_ub = signal_power/(isi_power + noise_power)
print("SNR_unbiased:", SNR_ub)
print("SNR_unbiased (dB)", 10*np.log10(SNR_ub))

SNR_biased: 10.761218135619552
SNR_biased (dB) 10.318614348651678
SNR_unbiased: 11.761218135619552
SNR_unbiased (dB) 10.704523049187294
